In [12]:
!pip install Pygments>=2.3.1

In [55]:
import re
import os.path
from markdown import markdown


class Renderer:
    def __init__(self, template_folder, template_path, content_path, output_path):
        self.template_path = template_path
        self.content_path = content_path
        self.template_folder = template_folder
        self.output_path = output_path
    
    
    def get_file_content(self, path):
        file = open(path, 'r') 
        content = file.read()
        file.close()
        return content
    
    
    def read_content(self):
        # collect variables now
        file = open(self.content_path, 'r')
        lines = file.readlines()
        file.close()
        
        # read lines, get the ones starting with
        context = {}
        content = []
        
        for line in lines:
            if line[0] == '@':
                name, val = line[1:].split(':')
                context[name] = val.strip()
            else:
                content.append(line)
                
        context['content'] = markdown(''.join(content), extensions=['codehilite'])
        
        return context
    
    
    def render_template(self):
        context = self.read_content()
        template_content = self.get_file_content(self.template_path)
        
        # find <%SYMBOL%>
        symbols = re.findall(r"<\%(.*?)\%>", template_content)
        symbols = [k.replace('<%', '').replace('%>', '') for k in symbols]
        
        # resolve symbols with variables
        for symbol in symbols:
            # if symbol is file
            file_path = f'{self.template_folder}\\{symbol}.html'
            if os.path.isfile(file_path):
                # read file content and replace
                template_content = template_content.replace(f'<%{symbol}%>', self.get_file_content(file_path))
            elif symbol in context:
                # get val from context
                template_content = template_content.replace(f'<%{symbol}%>', context[symbol])
            else:
                # throw error
                raise Exception(f'Symbol cannot be resolved. No file "{file_path}", No variable in content as "{symbol}"')
        
        return template_content
    
    
    def save_html(self):
        file = open(self.output_path, 'w')
        file.write(self.render_template())
        file.close()

In [58]:
# render one post
Renderer(
    template_folder = '.\\template_default',
    template_path   = '.\\template_default\\post.html', 
    content_path    = '.\\posts\\datalake-paths.md',
    output_path     = '.\\output\datalake-paths.html'
).save_html()

In [63]:
class IndexRenderer(Renderer):
    pass

In [64]:
c = IndexRenderer(
    template_folder = '.\\template_default',
    template_path   = '.\\template_default\\post.html', 
    content_path    = '.\\posts\\datalake-paths.md',
    output_path     = '.\\output\datalake-paths.html'
)

In [ ]:
# render index
IndexRenderer(
    template_folder = '.\\template_default',
    template_path   = '.\\template_default\\index.html', 
    content_path    = '.\\posts',
    output_path     = '.\\output\index.html'
).save_html()